In [2]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [4]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [5]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [6]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [7]:
x=np.empty((12*471,9*18))
y=np.empty((12*471,1))

In [8]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=year_data[m][:,g:g+9].reshape(1,-1)
        y[m*471+g:m*471+g+1,:]=year_data[m][9,g+9]

- just test no normalize

In [9]:
import torch 
from torch import optim,nn

In [10]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [11]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [12]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [13]:
x1=torch.Tensor(x)
y1=torch.Tensor(y)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],512),nn.Linear(512,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

### just test data col normalize

In [ ]:
x.shape

In [14]:
x2=pd.DataFrame(x)
x2=x2.apply(lambda x:((x-x.mean())/x.std()))

In [15]:
x22=torch.Tensor(x2.values)
y22=torch.Tensor(y)

In [17]:
x2.describe()

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
count,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,...,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03,5.652000e+03
mean,2.065895e-15,1.361261e-16,-4.041903e-15,-2.625603e-15,2.080313e-15,-4.369942e-15,-1.373204e-15,-2.386626e-15,-4.881681e-16,-4.224199e-13,...,2.393108e-15,-5.601794e-16,-1.593834e-16,-6.258067e-16,1.009258e-16,-7.327826e-16,4.103032e-16,3.857396e-16,1.155409e-15,-1.405457e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.542380e+00,-5.541477e+00,-5.539834e+00,-5.537931e+00,-5.535965e+00,-5.533520e+00,-5.530868e+00,-5.528037e+00,-5.524426e+00,-1.511159e+01,...,-2.160628e+00,-1.607958e+00,-1.608443e+00,-1.607113e+00,-1.605401e+00,-1.605757e+00,-1.607268e+00,-1.607673e+00,-1.607926e+00,-1.609448e+00
25%,-7.217457e-01,-7.225582e-01,-7.232489e-01,-7.235381e-01,-7.235989e-01,-7.233135e-01,-7.229693e-01,-7.224626e-01,-7.215030e-01,-8.133217e-01,...,-7.552632e-01,-7.636022e-01,-7.641024e-01,-7.633663e-01,-7.625531e-01,-7.628962e-01,-7.639013e-01,-7.643024e-01,-7.648141e-01,-7.657381e-01
50%,7.373852e-02,7.264301e-02,7.156720e-02,7.091615e-02,7.052099e-02,7.045002e-02,7.041341e-02,7.053651e-02,7.105850e-02,-1.897329e-02,...,-1.931174e-01,-2.006981e-01,-2.012089e-01,-2.008684e-01,-2.006544e-01,-2.009889e-01,-2.016567e-01,-2.020554e-01,-2.027394e-01,-2.032647e-01
75%,7.101259e-01,7.088040e-01,7.074201e-01,7.064796e-01,7.058169e-01,7.054608e-01,7.051196e-01,7.049358e-01,7.051077e-01,7.753752e-01,...,5.564103e-01,5.498407e-01,5.493158e-01,5.491288e-01,5.485438e-01,5.482208e-01,5.480028e-01,5.476072e-01,5.466935e-01,5.466999e-01
max,2.141998e+00,2.140166e+00,2.138089e+00,2.136497e+00,2.135233e+00,2.134235e+00,2.133208e+00,2.132334e+00,2.131718e+00,2.364072e+00,...,5.053576e+00,4.959256e+00,4.958648e+00,4.955362e+00,4.950084e+00,4.949828e+00,4.952252e+00,4.951875e+00,4.949612e+00,4.952742e+00


In [18]:
x2.shape

(5652, 162)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x22.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x22,y22,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x22.shape[1],512),nn.Linear(512,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x22,y22,256,0,0)

- just test linear model

In [19]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [20]:
n_folds=5

def rmlse_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x)
    rmse=np.sqrt(-cross_val_score(model,x,y,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [21]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
score=rmlse_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


Lasso score: 5.9364 (0.6516)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [22]:
np.sqrt(34.326048 )

5.8588435719005165

In [23]:
enet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9))
scoreo=rmlse_cv(enet)
print("\n enet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


 enet score: 5.9364 (0.6516)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [24]:
ridge=make_pipeline(RobustScaler(),Ridge(alpha=0.5))
score=rmlse_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


 ridge score: 5.9404 (0.6500)



In [25]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)
score = rmlse_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Gradient Boosting score: 6.9629 (1.0416)



In [26]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmlse_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[15:41:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:41:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:42:01] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [27]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmlse_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

LGBM score: 6.6617 (0.9785)



In [28]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data[i]))
all_train_data=pd.concat(month_list,axis=1).T

In [29]:
month_list[11].shape

(18, 480)

In [30]:
all_train_data.shape

(5760, 18)

- check reshape data is right ?

In [31]:
all_train_data.iloc[-5:,9:]

,9,10,11,12,13,14,15,16,17
475,37.0,0.0,76.0,2.6,1.9,109.0,97.0,1.0,1.4
476,28.0,0.0,80.0,2.2,1.9,108.0,107.0,1.7,1.3
477,17.0,0.0,82.0,2.3,1.9,114.0,118.0,1.5,1.6
478,24.0,0.0,84.0,2.3,2.0,108.0,100.0,2.0,1.8
479,29.0,0.0,84.0,2.3,2.0,109.0,105.0,2.0,2.0


In [32]:
all_train_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.527413,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.290152,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,-12.300000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


### correct outlier value

In [33]:
year_data1=year_data.copy()

In [34]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [53]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [54]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534201,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.576035,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [55]:
for idx in [1,2,4,6,9,12,13]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >=0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 mon

In [56]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [57]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534201,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.576035,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [58]:
x3=np.empty((12*471,18*9))
y3=np.empty((12*471,1))

In [59]:
for m in range(12):
    for g in range(471):
        x3[m*471+g:m*471+g+1,:]=year_data1[m][:,g:g+9].reshape(1,-1)
        y3[m*471+g:m*471+g+1,:]=year_data1[m][9,g+9]

In [60]:
year_data[11][-5,:5]

array([1.7, 1.7, 1.7, 1.7, 1.8])

In [61]:
year_data1[11][-5,:5]

array([1.7, 1.7, 1.7, 1.7, 1.8])

In [62]:
x[-5,-5:]

array([1.3, 1.7, 0.7, 0.4, 1.1])

In [63]:
x3[-5,-5:]

array([1.3, 1.7, 0.7, 0.4, 1.1])

In [64]:
y[-5:]

array([[37.],
       [28.],
       [17.],
       [24.],
       [29.]])

In [65]:
y3[-5:]

array([[37.],
       [28.],
       [17.],
       [24.],
       [29.]])

In [66]:
y[y<0]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [67]:
y3[y3<0]

array([], dtype=float64)

In [68]:
x3[x3<0]

array([], dtype=float64)

In [69]:
x33=pd.DataFrame(x3)
x33=x33.apply(lambda x: (x-x.mean())/x.std())
x33=x33.values
y33=y3

In [52]:
x333=torch.Tensor(x33)
y333=torch.Tensor(y33)

In [70]:
y33[:5]

array([[30.],
       [41.],
       [44.],
       [33.],
       [37.]])

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,10000,0.00001,5,x1,y1,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],512),nn.Linear(512,256),nn.Linear(256,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],8),nn.Linear(8,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x1,y1,256,0,0)

In [ ]:
n_folds=5

def rmlse_cv2(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x33)
    rmse=np.sqrt(-cross_val_score(model,x33,y33,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [ ]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.005,random_state=1))
score=rmlse_cv2(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

- use boxcox transformer to gaussian dist

In [73]:
from scipy.stats import boxcox,boxcox_normmax
from scipy.special import boxcox1p

In [74]:
all_train_data1.shape

(5760, 18)

In [75]:
all_train_data1.shape[1]

18

In [76]:
for i in range(all_train_data1.shape[1]):
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
1 --skew :  -5.095255676523535 --kurt :  85.70760213466164
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
4 --skew :  5.378836580918865 --kurt :  41.25921290333162
5 --skew :  1.6391191931961593 --kurt :  3.8457481859587697
6 --skew :  2.071649558193799 --kurt :  6.419180514549189
7 --skew :  1.3121217461933794 --kurt :  5.2975502420733696
8 --skew :  1.103678593463204 --kurt :  1.7728459308764268
9 --skew :  1.2763236528649795 --kurt :  2.115391100082106
10 --skew :  21.7839204262489 --kurt :  623.8917732792679
11 --skew :  -0.6050425772235472 --kurt :  -0.1979575284081454
12 --skew :  2.1806637991938813 --kurt :  8.332926208370576
13 --skew :  -0.7973228533500623 --kurt :  21.5544747714386
14 --skew :  0.5569055078560654 --kurt :  -0.9777533530993172
15 --skew :  0.5166061431100419 --kurt :  -1.0455161114586577
16 --skew :  0.9251663396039612 --kurt :  1

In [77]:
all_train_data2=pd.DataFrame()
for i in range(all_train_data1.shape[1]):
    lm=boxcox_normmax(all_train_data1[i]+1)
    all_train_data2[i]=boxcox1p(all_train_data1[i],lm)
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())
    print('lambda : ',lm)
    print(all_train_data2.index[i],'--skew : ',all_train_data2[i].skew(),'--kurt : ',all_train_data2[i].kurt())
    print('--------------------------------------------------------------------')

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
lambda :  1.461645538288768
0 --skew :  -0.07414275326700835 --kurt :  -0.9258821506244836
--------------------------------------------------------------------
1 --skew :  -5.095255676523535 --kurt :  85.70760213466164
lambda :  5.990940784090655
1 --skew :  0.2052435896115669 --kurt :  0.583625236412292
--------------------------------------------------------------------
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
lambda :  -3.259330902857795
2 --skew :  -0.003907562716963148 --kurt :  -0.06972890461562153
--------------------------------------------------------------------
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
lambda :  -6.182620739009405
3 --skew :  0.05859262861967747 --kurt :  -0.5568735662256521
--------------------------------------------------------------------
4 --skew :  5.378836580918865 --kurt :  41.25921290333162
lambda :  -0.7826645126628972
4 --skew :  -0.03484059806052529 --

In [78]:
all_train_data2.shape

(5760, 18)

In [79]:
year_data2={}
for m in range(12):
    month_data2=np.empty((18,480))
    month_data2[:,:]=all_train_data2.T.iloc[:,m*480:(m+1)*480]
    year_data2[m]=month_data2
    

In [80]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data1.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

In [81]:
year_data[11][-6,-5:]

array([2.6, 2.2, 2.3, 2.3, 2.3])

In [82]:
year_data3[11][-6,-5:]

array([2.6, 2.2, 2.3, 2.3, 2.3])

In [83]:
year_data2[11][-6,-5:]

array([1.1003593 , 1.01289662, 1.03598957, 1.03598957, 1.03598957])

### compare all_train_data1 and all_train_data2
- all_train_data1 : just correct outlier value
- all_train_data2 : just boxcox after all_train_data1

In [84]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534201,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.576035,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [85]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,65.832202,0.186377,0.079806,0.672361,2.283694,1.937404,7.449352,9.327724,5.756380,0.008977,15482.226219,1.051691,3.676586,12.803597,11.687312,1.046624,0.813470
std,25.536335,12.405338,0.041562,0.032614,0.176061,0.523410,0.306804,2.431622,3.291614,2.725299,0.035324,6063.422584,0.303361,0.481140,3.602422,3.020925,0.263948,0.292340
min,12.833239,0.000000,0.068067,0.000000,0.000000,0.000000,0.769032,0.000000,0.000000,0.000000,0.000000,1616.845054,0.000000,0.000000,0.096879,0.000000,0.000000,0.000000
25%,49.926550,50.952479,0.158556,0.055291,0.562804,1.938097,1.735287,5.766358,7.204513,3.853588,0.000000,10617.557930,0.852464,3.245618,9.751980,9.184518,0.857234,0.592611
50%,70.525274,63.921527,0.188618,0.076901,0.672848,2.270782,1.916995,7.472206,9.283713,5.763730,0.000000,15534.166404,1.035990,3.534372,12.099014,11.107693,1.042283,0.818152
75%,88.521007,79.522794,0.215417,0.103613,0.775804,2.651342,2.143833,9.063998,11.523345,7.603404,0.000000,20398.022735,1.243713,3.834142,15.383128,13.963600,1.233421,1.030980
max,133.380091,120.311749,0.306532,0.160805,1.192886,3.876332,2.905076,20.678088,20.721569,14.709463,0.161645,30295.187454,2.192628,7.863631,18.965279,16.819218,1.852950,1.618322


## diff with hw_test81 begin
- all_train_data3 : normalized
- all_train_data2 : no normalized

In [ ]:
# (all_train_data2[0] - all_train_data2[0].mean()) / all_train_data2[0].std()

In [86]:
all_train_data3=pd.DataFrame()
for i in range(all_train_data2.shape[1]):
    if i!=9:
        all_train_data3[i]=(all_train_data2[i] - all_train_data2[i].mean()) / all_train_data2[i].std()
    else:
        all_train_data3[i]=all_train_data2[i]

In [87]:

# all_train_data3=all_train_data3.apply(lambda x : (x-x.mean())/x.std())

In [88]:
all_train_data3.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5760.000000,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,1.019875e-13,-5.144712e-14,6.653925e-14,2.549689e-14,2.334629e-15,-6.727368e-14,-4.221841e-14,2.062856e-15,5.215490e-14,5.756380,-1.659610e-15,6.245795e-14,-9.682569e-15,-1.537578e-14,1.616595e-14,-8.086471e-15,3.354384e-14,3.181464e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.725299,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.255345e+00,-5.306764e+00,-2.846587e+00,-2.447008e+00,-3.818917e+00,-4.363104e+00,-3.808196e+00,-3.063532e+00,-2.833784e+00,0.000000,-2.541260e-01,-2.286725e+00,-3.466795e+00,-7.641412e+00,-3.527271e+00,-3.868786e+00,-3.965265e+00,-2.782619e+00
25%,-8.027755e-01,-1.199461e+00,-6.693840e-01,-7.516859e-01,-6.222681e-01,-6.602794e-01,-6.587790e-01,-6.921282e-01,-6.450364e-01,3.853588,-2.541260e-01,-8.022974e-01,-6.567335e-01,-8.957229e-01,-8.471015e-01,-8.284860e-01,-7.175293e-01,-7.554874e-01
50%,3.868283e-03,-1.540203e-01,5.390355e-02,-8.905258e-02,2.767230e-03,-2.466757e-02,-6.652185e-02,9.398760e-03,-1.337080e-02,5.763730,-2.541260e-01,8.566150e-03,-5.175865e-02,-2.955772e-01,-1.955858e-01,-1.918681e-01,-1.644871e-02,1.601343e-02
75%,7.085792e-01,1.103605e+00,6.987095e-01,7.299705e-01,5.875442e-01,7.024097e-01,6.728357e-01,6.640201e-01,6.670347e-01,7.603404,-2.541260e-01,8.107297e-01,6.329803e-01,3.274645e-01,7.160547e-01,7.535070e-01,7.077023e-01,7.440291e-01
max,2.465256e+00,4.391621e+00,2.890982e+00,2.483615e+00,2.956513e+00,3.042810e+00,3.154037e+00,5.440292e+00,3.461476e+00,14.709463,4.321989e+00,2.443003e+00,3.760984e+00,8.702351e+00,1.710428e+00,1.698787e+00,3.054866e+00,2.753135e+00


In [89]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,65.832202,0.186377,0.079806,0.672361,2.283694,1.937404,7.449352,9.327724,5.756380,0.008977,15482.226219,1.051691,3.676586,12.803597,11.687312,1.046624,0.813470
std,25.536335,12.405338,0.041562,0.032614,0.176061,0.523410,0.306804,2.431622,3.291614,2.725299,0.035324,6063.422584,0.303361,0.481140,3.602422,3.020925,0.263948,0.292340
min,12.833239,0.000000,0.068067,0.000000,0.000000,0.000000,0.769032,0.000000,0.000000,0.000000,0.000000,1616.845054,0.000000,0.000000,0.096879,0.000000,0.000000,0.000000
25%,49.926550,50.952479,0.158556,0.055291,0.562804,1.938097,1.735287,5.766358,7.204513,3.853588,0.000000,10617.557930,0.852464,3.245618,9.751980,9.184518,0.857234,0.592611
50%,70.525274,63.921527,0.188618,0.076901,0.672848,2.270782,1.916995,7.472206,9.283713,5.763730,0.000000,15534.166404,1.035990,3.534372,12.099014,11.107693,1.042283,0.818152
75%,88.521007,79.522794,0.215417,0.103613,0.775804,2.651342,2.143833,9.063998,11.523345,7.603404,0.000000,20398.022735,1.243713,3.834142,15.383128,13.963600,1.233421,1.030980
max,133.380091,120.311749,0.306532,0.160805,1.192886,3.876332,2.905076,20.678088,20.721569,14.709463,0.161645,30295.187454,2.192628,7.863631,18.965279,16.819218,1.852950,1.618322


In [90]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data3.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

## diff with hw_test81 end

In [ ]:
x4=np.empty((12*471,18*9))
y4=np.empty((12*471,1))

for m in range(12):
    for g in range(471):
        x4[m*471+g:m*471+g+1,:]=year_data3[m][:,g:g+9].reshape(1,-1)
        y4[m*471+g:m*471+g+1,:]=year_data3[m][9,g+9]

In [ ]:
x4[-6,81:92]

In [ ]:
x4[-5,81:92]

In [ ]:
x4[-4,81:92]

In [ ]:
y4.shape

In [ ]:
y4[-6:]

In [ ]:
y4[-5:]

In [ ]:
y4[-4:]

In [ ]:
x44=torch.Tensor(x4)
y44=torch.Tensor(y4)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],128),nn.ReLU(),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,5000,0.00001,5,x44,y44,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],256),nn.Linear(256,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],16),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,1000,0.0005,5,x44,y44,256,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],32),nn.Linear(32,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

#### set lr = 0.0001 , batch_size = 64 , test loss <=1

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],16),nn.ReLU(),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x44,y44,64,0,0)

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],8),nn.ReLU(),nn.Linear(8,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x44,y44,64,0,0)

### linear function  test

In [ ]:
def rmsle_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x4)
    rmse=np.sqrt(-cross_val_score(model,x4,y4,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [ ]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
score=rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
enet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9))
scoreo=rmsle_cv(enet)
print("\n enet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
ridge=make_pipeline(RobustScaler(),Ridge(alpha=0.5))
score=rmsle_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
ridge=make_pipeline(Ridge(alpha=0.5))
score=rmsle_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
# x4_cxv=pd.DataFrame(x4.reshape(-1,9))
# x4_cxv.to_csv(r'F:\study\ml\HonyiLee2020\hw1\data\wx_x_cxv.csv')

In [ ]:
x4[-5,81:91]

In [ ]:
x4[-4,81:91]

In [ ]:
x4[-3,81:91]

In [ ]:
y4[-5:]

In [ ]:
all_train_data3.describe()

In [ ]:
 0.6281137*2.725299+5.756380

In [ ]:
from scipy.special import inv_boxcox1p

In [ ]:
 0.6281137*2.725299+5.756380
inv_boxcox1p(7.4681776384963,0.4145678510523434)

In [ ]:
-0.06379855*2.725299+5.756380

In [ ]:
inv_boxcox1p(5.5825098754835505,0.4145678510523434)

In [ ]:
0.38*0.38

In [ ]:
x4_len=x4.shape[0]
print(int(0.8*x4_len))
x4_train=x4[:int(0.8*x4_len),:]
y4_train=y4[:int(0.8*x4_len),:]
x4_test=x4[int(0.8*x4_len):,:]
y4_test=y4[int(0.8*x4_len):,:]

In [ ]:
x4_train.shape

In [ ]:
y4_train.shape

In [ ]:
lasso=Lasso(alpha=0.0005,random_state=1)
lasso.fit(x4_train,y4_train)
y4_hat=lasso.predict(x4_test)

In [ ]:
y4_hat[-5:]

In [ ]:
y4_test[-5:]

In [ ]:
0.04508735*2.725299+5.756380

inv_boxcox1p(5.87925650986765,0.4145678510523434)

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
def rmsle_cv_predict(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x4_train)
    rmse=np.sqrt(-cross_val_predict(model,x4_train,y4_train,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [ ]:
lasso=Lasso(alpha=0.0005,random_state=1)
lasso.fit(x4_train,y4_train)
y4_hat=lasso.predict(x4_test).reshape(-1,1)
np.sqrt(np.sum(np.power(y4_hat-y4_test,2),axis=0)/y4_test.shape[0])

In [ ]:
y4_hat_orig=y4_hat
y4_test_orig=y4_test
y4_hat_orig=inv_boxcox1p(y4_hat_orig,0.4145678510523434)
y4_test_orig=inv_boxcox1p(y4_test_orig,0.4145678510523434)
np.sqrt(mean_squared_error(y4_hat_orig,y4_test_orig))

In [ ]:
inv_boxcox1p(1.05,0.4145678510523434)

In [ ]:
ridge=Ridge(alpha=0.5)
ridge.fit(x4_train,y4_train)
y_hat=ridge.predict(x4_test)

In [ ]:
y_hat[-5:]

In [ ]:
x44_train=torch.Tensor(x4_train)
y44_train=torch.Tensor(y4_train)
x44_test=torch.Tensor(x4_test)

In [ ]:
def train_kfold_pred(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd,test_features):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')
    preds=net(test_features).detach().numpy()
    print('pred :\n', preds[-5:])

    return preds

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],8),nn.ReLU(),nn.Linear(8,1))
net1=get_net()
y4_hat=train_kfold_pred(net1,5000,0.0001,5,x44_train,y44_train,64,0,0,x44_test)